In [2]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os
from zipfile import ZipFile
from os import listdir
from os.path import isfile, join
import datetime
import time
import os
import numpy as np
import tensorflow as tf
from pathlib import Path
from tokenizers import Tokenizer, models, trainers
from os import listdir
from os.path import isfile, join

In [7]:
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

In [308]:
path = 'data/firstratedata/'
zipfiles = [f'crypto-active_1min_1nfzd.zip', f'crypto-active_5min_ziyhm.zip', 
            f'crypto-active_30min_wnk5g.zip', 
            f'crypto-active_1hour_9tsmq.zip']
dayOfWeek={0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
samples_tokenized = set([])
for zipfile in zipfiles:
    zf = ZipFile(f'{path+zipfile}') 
    data_files = zf.filelist
    pbar = tqdm(enumerate(data_files), total=len(data_files))
    for idx, file in pbar:
        if file.filename.startswith('USDC') or file.filename.startswith('USDT') or file.filename.startswith('BTC-EUR') or file.filename.startswith('ETH-BTC') or file.filename.startswith('UST'):
            continue
        pbar.set_postfix(file=file.filename)
        df = pd.read_csv(zf.open(file), names=['datetime', 'open', 'high', 'low', 'close', 'volume'])
        df['datetime'] = pd.to_datetime(df['datetime'])
        df['weekday'] = df['datetime'].dt.dayofweek.map(dayOfWeek)
        grid = (df['datetime'].dt.year == 2021) & (df['datetime'].dt.month == 7)
        df = df.round(6)
        
        df_train = df[~grid]
        df_val = df[grid]
        
        coin = file.filename.split('_')[0]
        tokens = ';'.join([f"crypt{row['datetime'].month}_{row['datetime'].day}{row['weekday']}{row['datetime'].time().isoformat()[:5]}O{row['open']}H{row['high']}L{row['low']}C{row['close']}V{int(row['volume'])}" for idx, row in df_train.iterrows()])
        
        text_file = open(f"strings/train/{file.filename[:-4]}_string.txt", "w")
        n = text_file.write(tokens)
        text_file.close()  
        
#         tokens = tokenizer.encode(tokens)
#         samples_tokenized.update(set(tokens))
        
        tokens = ';'.join([f"crypt{row['datetime'].month}_{row['datetime'].day}{row['weekday']}{row['datetime'].time().isoformat()[:5]}O{row['open']}H{row['high']}L{row['low']}C{row['close']}V{int(row['volume'])}" for idx, row in df_val.iterrows()])
        text_file = open(f"strings/val/{file.filename[:-4]}_string.txt", "w")
        n = text_file.write(tokens)
        text_file.close()  

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [03:03<00:00,  3.46s/it, file=ZRX_1hour.txt]


In [33]:
new_vocab = {}

f = open('merges.txt', 'r', encoding="utf8")
lines = f.readlines()
lines[1:]
f.close()
with open('gpt_subset_tokenizer/merges.txt', 'w') as filehandle:
    for line in lines:
        t = line.replace(' ', '').replace('\n', '')
        if t.isdigit():
            filehandle.write(line)
            words = line.replace('\n', '').split(' ')
            words += [''.join(words)]
            for w, ID in zip(words, tokenizer.convert_tokens_to_ids(words)):
                new_vocab[w] = ID

In [34]:
new_words = [
    'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun',
    'O', 'H', 'L', 'C', 'V',
    'crypt', ';', '.', ':', '_'
]

for w, ID in zip(new_words, tokenizer.convert_tokens_to_ids(new_words)):  # for name, age in dictionary.iteritems():  (for Python 2.x)
    new_vocab[w] = ID

In [17]:
len(new_vocab)

NameError: name 'new_vocab' is not defined

In [38]:
import json
json.dump(new_vocab, open('gpt_subset_tokenizer/vocab_mine.json', 'w'))

In [10]:
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2', progress=False)

In [12]:
tokenizer_new = GPT2TokenizerFast('gpt_subset_tokenizer/vocab.json', 'gpt_subset_tokenizer/merges.txt')

In [14]:
tokenizer_new = GPT2TokenizerFast.from_pretrained('gpt_subset_tokenizer')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
tokenizer_new.vocab['crypt']

29609

In [15]:
t = "crypt"
tokenizer.encode(t), tokenizer_new.encode(t)

([29609], [])

In [25]:
tokenizer.encode('V1')

[53, 16]

In [18]:
tokenizer.encode('crypt7_1Thu00:00O35060.0H35071.34L34985.47C34995.83V20;crypt7_1Thu00:01O34995.83H35049.37L34971.47C34992.57V14;crypt7_1Thu00:02O34988.11H35014.53L34931.08C34961.2V75;crypt7_1Thu00:03O34967.9H35005.65L34954.13C34961.23V27;crypt7_1Thu00:04O34962.6H35010.7L34962.59C34983.38V10;crypt7_1Thu00:05O34971.31H34983.37L34904.51C34905.03V13;crypt7_1Thu00:06O34905.05H34939.0L34900.01C34930.8V19;crypt7_1Thu00:07O34912.38H34976.0L34910.43C34935.3V29;crypt7_1Thu00:08O34936.23H34979.95L34918.74C34979.95V14;crypt')

[29609,
 22,
 62,
 16,
 39902,
 405,
 25,
 405,
 46,
 14877,
 1899,
 13,
 15,
 39,
 14877,
 4869,
 13,
 2682,
 43,
 2682,
 42250,
 13,
 2857,
 34,
 2682,
 33438,
 13,
 5999,
 53,
 1238,
 26,
 29609,
 22,
 62,
 16,
 39902,
 405,
 25,
 486,
 46,
 2682,
 33438,
 13,
 5999,
 39,
 14877,
 2920,
 13,
 2718,
 43,
 27371,
 4869,
 13,
 2857,
 34,
 2682,
 41561,
 13,
 3553,
 53,
 1415,
 26,
 29609,
 22,
 62,
 16,
 39902,
 405,
 25,
 2999,
 46,
 27371,
 3459,
 13,
 1157,
 39,
 2327,
 28645,
 13,
 4310,
 43,
 27371,
 3132,
 13,
 2919,
 34,
 2682,
 4846,
 16,
 13,
 17,
 53,
 2425,
 26,
 29609,
 22,
 62,
 16,
 39902,
 405,
 25,
 3070,
 46,
 27371,
 3134,
 13,
 24,
 39,
 2327,
 22544,
 13,
 2996,
 43,
 2682,
 48372,
 13,
 1485,
 34,
 2682,
 4846,
 16,
 13,
 1954,
 53,
 1983,
 26,
 29609,
 22,
 62,
 16,
 39902,
 405,
 25,
 3023,
 46,
 2682,
 4846,
 17,
 13,
 21,
 39,
 2327,
 20943,
 13,
 22,
 43,
 2682,
 4846,
 17,
 13,
 3270,
 34,
 2682,
 4089,
 18,
 13,
 2548,
 53,
 940,
 26,
 29609,
 22,
 62,
 16,


In [41]:
tokenizer_new.convert_ids_to_tokens(tokenizer_new.encode(t))

['110']

In [50]:
tokenizer_new.vocab_size, tokenizer.vocab_size

(1008, 50257)

In [26]:
train_done_files = [f[:-4] for f in listdir('tokenized_subset_data/train/')]
val_done_files = [f[:-4] for f in listdir('tokenized_subset_data/val/')]

train_files = [f for f in listdir('strings/train/') if f[:-11] not in train_done_files]
val_files = [f for f in listdir('strings/val/') if f[:-11] not in val_done_files]

In [27]:
%%time
for frame in ['_1min', '5min', '_30min', '_1hour']:
    print(frame)
    for file in train_files:
        if frame in file:
            samples_tokenized2 = []
            print(file)
            f = Path('strings/train/'+file).read_text()
            tokens = tokenizer.encode(f)
            for i in range(0, len(tokens), 2048):
                samples_tokenized2.append(tokens[i:i+2048])
            del samples_tokenized2[-1]
            
            samples_tokenized2 = np.array(samples_tokenized2)
            np.save(f'tokenized_subset_data/train/{file[:-11]}', samples_tokenized2)

_1min
ADA_1min_string.txt


Token indices sequence length is longer than the specified maximum sequence length for this model (37372178 > 1024). Running this sequence through the model will result in indexing errors


BAT_1min_string.txt
BCH_1min_string.txt
BNT_1min_string.txt
BSV_1min_string.txt
BTC_1min_string.txt
BTG_1min_string.txt
CVC_1min_string.txt
DAI_1min_string.txt
DASH_1min_string.txt
DCR_1min_string.txt
DOGE_1min_string.txt
EOS_1min_string.txt
ETC_1min_string.txt
ETH_1min_string.txt
FUN_1min_string.txt
HT_1min_string.txt
ICX_1min_string.txt
IOST_1min_string.txt
KNC_1min_string.txt
LINK_1min_string.txt
LRC_1min_string.txt
LSK_1min_string.txt
LTC_1min_string.txt
MAID_1min_string.txt
MANA_1min_string.txt
MKR_1min_string.txt
NEO_1min_string.txt
OMG_1min_string.txt
ONT_1min_string.txt
PAX_1min_string.txt
QTUM_1min_string.txt
REP_1min_string.txt
SC_1min_string.txt
SNT_1min_string.txt
TRX_1min_string.txt
UTK_1min_string.txt
VET_1min_string.txt
WAVES_1min_string.txt
XEM_1min_string.txt
XLM_1min_string.txt
XMR_1min_string.txt
XRP_1min_string.txt
XTZ_1min_string.txt
XVG_1min_string.txt
ZEC_1min_string.txt
ZIL_1min_string.txt
ZRX_1min_string.txt
5min
ADA_5min_string.txt
BAT_5min_string.txt
BCH_5min

In [28]:
for frame in ['_1min', '5min', '_30min', '_1hour']:
    print(frame)
    for file in val_files:
        if frame in file:
            samples_tokenized2 = []
            print(file)
            f = Path('strings/val/'+file).read_text()
            tokens = tokenizer.encode(f)
            for i in range(0, len(tokens), 2048):
                samples_tokenized2.append(tokens[i:i+2048])
            if len(samples_tokenized2) > 0:
                del samples_tokenized2[-1]
            
            samples_tokenized2 = np.array(samples_tokenized2)
            np.save(f'tokenized_subset_data/val/{file[:-11]}', samples_tokenized2)

_1min
ADA_1min_string.txt
BAT_1min_string.txt
BCH_1min_string.txt
BNT_1min_string.txt
BSV_1min_string.txt
BTC_1min_string.txt
BTG_1min_string.txt
CVC_1min_string.txt
DAI_1min_string.txt
DASH_1min_string.txt
DCR_1min_string.txt
DOGE_1min_string.txt
EOS_1min_string.txt
ETC_1min_string.txt
ETH_1min_string.txt
FUN_1min_string.txt
HT_1min_string.txt
ICX_1min_string.txt
IOST_1min_string.txt
KNC_1min_string.txt
LINK_1min_string.txt
LRC_1min_string.txt
LSK_1min_string.txt
LTC_1min_string.txt
MAID_1min_string.txt
MANA_1min_string.txt
MKR_1min_string.txt
NEO_1min_string.txt
OMG_1min_string.txt
ONT_1min_string.txt
PAX_1min_string.txt
QTUM_1min_string.txt
REP_1min_string.txt
SC_1min_string.txt
SNT_1min_string.txt
TRX_1min_string.txt
UTK_1min_string.txt
VET_1min_string.txt
WAVES_1min_string.txt
XEM_1min_string.txt
XLM_1min_string.txt
XMR_1min_string.txt
XRP_1min_string.txt
XTZ_1min_string.txt
XVG_1min_string.txt
ZEC_1min_string.txt
ZIL_1min_string.txt
ZRX_1min_string.txt
5min
ADA_5min_string.txt
BA

In [29]:
def _int64_feature(value):
    """
    Returns an int64_list from a bool / enum / int / uint.
    """
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def write_to_file(writer, data):
    """
    writes data to tfrecord file
    """
    feature = {
        "text": _int64_feature(data)
    }
    tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
    writer.write(tf_example.SerializeToString())
    
    
def split_list(l, n):
    # splits list/string into n size chunks
    return [l[i:i + n] for i in range(0, len(l), n)]


def write_files(files, files_per, output_dir, out_name, start_no, write_remainder=True, process_no=None):
    # writes a list of files to .tfrecords
    chunks = split_list(files, files_per)

    if len(chunks[-1]) != files_per and not write_remainder:  # pop the last file if it's length != files per
        remainder = chunks.pop(-1)
    else:
        remainder = None  # assuming files = remainder from an old chunk here
        files_per = len(chunks[-1])

    for files in chunks:
        fp = f"{output_dir}/{out_name}_{start_no}"
        if process_no is not None:
            fp += f"_{process_no}"
        fp += f"_{files_per}"  # add number of files in tfrecord to end of fp
        fp += ".tfrecords"
        with tf.io.TFRecordWriter(fp) as writer:
            for f in files:
                write_to_file(writer, f)
        start_no += 1
    return start_no, remainder


def create_tfrecords(tokenized_files_array, output_dir, out_name):
    files_per = 100000
    _tfrecord_count, remainder = write_files(tokenized_files_array, files_per=files_per,
                                                         output_dir=output_dir, out_name=out_name,
                                                         start_no=0, process_no=None)
    
@tf.function
def decode_fn(record_bytes):
  return tf.io.parse_single_example(
      # Data
      record_bytes,

      # Schema
      {"x": tf.io.FixedLenFeature([], dtype=tf.float32),
       "y": tf.io.FixedLenFeature([], dtype=tf.float32)}
  )

In [30]:
samples_tokenized3 = np.concatenate([np.load(f'tokenized_subset_data/train/{file[:-11]}.npy') for file in train_files])
results = create_tfrecords(samples_tokenized3, 'records_subset', 'crypto_train')

In [31]:
samples_tokenized3 = np.concatenate([np.load(f'tokenized_subset_data/val/{file[:-11]}.npy') for file in val_files if np.load(f'tokenized_subset_data/val/{file[:-11]}.npy').shape[0] > 0])
results = create_tfrecords(samples_tokenized3, 'records_subset', 'crypto_val')